In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os

os.environ['HF_HOME'] = 'D:/NLP/huggingface_cache'

In [2]:
load_dotenv()

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [8]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id='meta-llama/Meta-Llama-3-8B-Instruct',
    task='text-generation'
)

model = ChatHuggingFace(llm=llm)

In [3]:
from langchain.schema import Document

# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [5]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.vectorstores import FAISS

In [6]:
vector_store = FAISS.from_documents(
    documents=docs,
    embedding=embeddings
)

In [9]:
compressor = LLMChainExtractor.from_llm(model)

In [11]:
retriever = ContextualCompressionRetriever(
    base_retriever=vector_store.as_retriever(search_kwargs={'k':2}),
    base_compressor=compressor
)

In [12]:
query = 'what is photosynthesis?'

res = retriever.invoke(query)

In [15]:
for i,doc in enumerate(res):
    print(doc.page_content)

Photosynthesis is the process by which green plants convert sunlight into energy.
The chlorophyll in plant cells captures sunlight during photosynthesis.
